# **Import Required Libraries**

In [2]:
import tensorflow as tf  # Import TensorFlow library
from tensorflow.keras import layers, models  # Import necessary Keras modules
import numpy as np  # Import NumPy for numerical operations
import matplotlib.pyplot as plt  # Import Matplotlib for visualization
from tensorflow.keras.utils import to_categorical  # Import function for one-hot encoding labels
from sklearn.model_selection import train_test_split  # Import function for splitting dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Import image data augmentation tool
from tensorflow.keras.models import load_model  # Import function to load a saved model
from PIL import Image  # Import Python Imaging Library for image processing
from google.colab import files  # Import Colab files module to upload images


ModuleNotFoundError: No module named 'tensorflow'

# **Load and Preprocess the MNIST Dataset**

In [7]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print(f"Training samples: {x_train.shape[0]}, Testing samples: {x_test.shape[0]}")  # Print dataset size

Training samples: 60000, Testing samples: 10000


In [8]:
# Resize images to 32x32
x_train_resized = np.array([tf.image.resize(img[..., np.newaxis], (32, 32)).numpy().squeeze() for img in x_train])
x_test_resized = np.array([tf.image.resize(img[..., np.newaxis], (32, 32)).numpy().squeeze() for img in x_test])

print(f"Resized training shape: {x_train_resized.shape}, Testing shape: {x_test_resized.shape}")  # Print resized shape

Resized training shape: (60000, 32, 32), Testing shape: (10000, 32, 32)


In [9]:
# Normalize images (pixel values between 0 and 1)
x_train_resized = x_train_resized / 255.0
x_test_resized = x_test_resized / 255.0

**Split Dataset into Training and Validation Sets**

In [10]:
# Split dataset: 80% training, 20% validation
x_train, x_val, y_train, y_val = train_test_split(x_train_resized, y_train, test_size=0.2, random_state=42)

print(f"Training shape: {x_train.shape}, Validation shape: {x_val.shape}, Testing shape: {x_test.shape}")  # Print final dataset shapes

Training shape: (48000, 32, 32), Validation shape: (12000, 32, 32), Testing shape: (10000, 28, 28)


In [11]:
# One-hot encode labels (convert labels into categorical format for multi-class classification)
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)
y_test = to_categorical(y_test, 10)

print(f"Sample one-hot encoded label: {y_train[0]}")  # Print an example one-hot encoded label

Sample one-hot encoded label: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


**Define Data Augmentation**

In [12]:
# Improved Data augmentation
data_augmentation = ImageDataGenerator(
    rotation_range=15,  # Increased rotation range
    width_shift_range=0.15,  # Increased shift range
    height_shift_range=0.15,
    shear_range=0.15,  # Shear transformation
    zoom_range=0.2  # Increased zoom
)

# **Build the CNN Model**



In [13]:
# Build improved CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1), kernel_regularizer=tf.keras.regularizers.l2(0.001)),  # L2 Regularization
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),  # Dropout to reduce overfitting
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])

# **Training and Compilation**

In [14]:
# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
# Train the model with data augmentation
model.fit(data_augmentation.flow(x_train[..., np.newaxis], y_train, batch_size=64),  # Apply data augmentation
          epochs=6,  # Reduced to 6 epochs
          validation_data=(x_val[..., np.newaxis], y_val))  # Validate using validation set

Epoch 1/6
750/750 ━━━━━━━━━━━━━━━━━━━━ 53s 67ms/step - accuracy: 0.5836 - loss: 1.4507 - val_accuracy: 0.9637 - val_loss: 0.3417
Epoch 2/6
750/750 ━━━━━━━━━━━━━━━━━━━━ 50s 67ms/step - accuracy: 0.8985 - loss: 0.5536 - val_accuracy: 0.9784 - val_loss: 0.2737
Epoch 3/6
750/750 ━━━━━━━━━━━━━━━━━━━━ 50s 67ms/step - accuracy: 0.9233 - loss: 0.4482 - val_accuracy: 0.9824 - val_loss: 0.2523
Epoch 4/6
750/750 ━━━━━━━━━━━━━━━━━━━━ 47s 63ms/step - accuracy: 0.9311 - loss: 0.4102 - val_accuracy: 0.9786 - val_loss: 0.2445
Epoch 5/6
750/750 ━━━━━━━━━━━━━━━━━━━━ 49s 66ms/step - accuracy: 0.9380 - loss: 0.3760 - val_accuracy: 0.9841 - val_loss: 0.2217
Epoch 6/6
750/750 ━━━━━━━━━━━━━━━━━━━━ 82s 66ms/step - accuracy: 0.9438 - loss: 0.3563 - val_accuracy: 0.9839 - val_loss: 0.2241


# **Evaluate Model Performance**

In [16]:
# Evaluate model performance
test_loss, test_acc = model.evaluate(x_test_resized[..., np.newaxis], y_test)  # Fixed test dataset shape
print(f"Test accuracy: {test_acc:.4f}")  # Print test accuracy

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9830 - loss: 0.2222
Test accuracy: 0.9867


# **Save and Load the Trained Model**

In [17]:
# Save the trained model
model.save("handwriting_cnn_model.h5")

In [18]:
# Load the saved model
loaded_model = load_model("handwriting_cnn_model.h5")

# **Predict Custom Handwritten Image**



In [19]:
# Function to predict a custom handwritten image
def predict_custom_image():
    uploaded = files.upload()  # Upload custom image
    for filename in uploaded.keys():
        image = Image.open(filename).convert('L')  # Convert image to grayscale
        image = image.resize((32, 32))  # Resize to 32x32 pixels
        image = np.array(image).astype('float32') / 255.0  # Normalize pixel values
        image = image.reshape((1, 32, 32, 1))  # Reshape for model input
        prediction = model.predict(image)  # Predict digit
        print(f'Predicted digit: {np.argmax(prediction)}')  # Print predicted digit

predict_custom_image()  # Call function to make a prediction

Saving image_2025-02-26_141848598.png to image_2025-02-26_141848598.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Predicted digit: 3
